# Get Ethnicity, Religion and Political Ideology Labels from Wikidata 

In [1]:
import requests
query_wikidata_labels = \
    'SELECT DISTINCT ?label WHERE {?item wdt:P31 ?type . {{?item rdfs:label ?label . FILTER(lang(?label) = "en")} '\
    'UNION {?item skos:altLabel ?label . FILTER(lang(?label) = "en")}}}'
item_dict = {':PoliticalIdeology': 'wd:Q12909644', ':Ethnicity': 'wd:Q41710', ':ReligiousBelief': 'wd:Q9174'}

In [2]:
for label_type in (':PoliticalIdeology', ':Ethnicity', ':ReligiousBelief'):
    new_ttl = ['@prefix : <urn:ontoinsights:dna:> .', '@prefix dna: <urn:ontoinsights:dna:> .',
               '@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .']
    labels = []
    query_labels = query_wikidata_labels.replace('?type', item_dict[label_type])
    response = requests.get(
        f'https://query.wikidata.org/sparql?format=json&query={query_labels}').json()
    if 'results' in response and 'bindings' in response['results']:
        results = response['results']['bindings']
        for result in results:
            labels.append(result['label']['value'].replace('"', "'"))
    for label in labels:
        if label.isascii() and len(label) > 3 and ' of ' not in label and ' in ' not in label and \
                ' people' not in label and ' culture' not in label:
            new_ttl.append(f'{label_type} :noun_synonym "{label}" .')
    with open(f'{label_type[1:].lower()}_synonyms.ttl', 'w')as outf:
        for next_ttl in new_ttl:
            outf.write(f'{next_ttl}\n')